In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pylab as plt
import cv2

In [ ]:
data_root = '/diskmnt/a/makov/yaivan/2016-02-11_Pin/'

Список файлов:
* empty - файл полученный с томографа без коррекций
* corr - то же изображение что и empty, но с коррекцией
* tomo - то же, что и empty, но полученное в ходе проведения эксперимента
* white - пустой пучок используемый для нормировки изображений (получен в тот-же день при калибровке)
* black_1, black_2 - темновые токи, полученные в разное время

In [ ]:
empty = plt.imread(data_root+'first_projection.tif').astype('float32')
corr = plt.imread(data_root+'first_projection_corr.tif').astype('float32')
tomo = plt.imread(data_root+'Raw/pin_2.24um_0000.tif').astype('float32')
white = np.fromfile(data_root+'white0202_2016-02-11.ffr',dtype='<u2').astype('float32').reshape((2096, 4000))
black_1 = np.fromfile(data_root+'black0101_2016-02-09.ffr',dtype='<u2').astype('float32').reshape((2096, 4000))
black_2 = np.fromfile(data_root+'black0201_2016-02-16.ffr',dtype='<u2').astype('float32').reshape((2096, 4000))

In [ ]:
def show_frame(data, label):
    data_filtered = cv2.medianBlur(data,5)
    plt.figure(figsize=(12,10))
    plt.imshow(data_filtered)
    plt.title(label+' filtered')
    plt.colorbar(orientation='horizontal')

    plt.show()
    plt.figure(figsize=(12,8))
    plt.plot(data[1000])
    plt.grid(True)
    plt.title(label+' filtered: central cut')
    plt.show()

    plt.figure(figsize=(12,10))
    plt.imshow(data_filtered)
    plt.colorbar(orientation='horizontal')
    plt.title(label)
    plt.show()
    plt.figure(figsize=(12,8))
    plt.plot(data_filtered[1000])
    plt.grid(True)
    plt.title(label+': central cut')
    plt.show()

## Вот пучок без объекта. 
По осям - отсчёты детектора.
Здесь и далее первая картинка и центральное сечение - как есть. Вторая картинка - с применениейм медианной фильтрации (чтобы убрать шумы сцинстиллятора).



In [ ]:
show_frame(white, 'White')

## Вот темновой ток 1
по осям - отсчёты детектора

In [ ]:
show_frame(black_1, 'Black_1')

## Вот темновой ток 2
по осям - отсчёты детектора

In [ ]:
show_frame(black_2, 'Black_2')

## Вот разница между темновыми токами

In [ ]:
show_frame(black_1 - black_2, 'Black_1 - Black_2')

## Вот никак не скорректированное изображение

In [ ]:
show_frame(empty, 'Empty')

## Вот отнормированное изображение (силами томографа)
Странно, что на центральном срезе максимум не на 65535 (2^16), а примерно 65535\*__0.8. Это значит что нам при реконструкции нужно нормироваться не на 65535 при взятии логарифма, а на максимум по синограмме?__

In [ ]:
show_frame(corr, 'Corr')

## Вот изображение из томографического зксперимента

In [ ]:
show_frame(tomo, 'tomo image')

## Вот разница изображений отнормированных томографом в ручном режиме и режиме томографа
Они видимо немого сдвинуты

In [ ]:
show_frame(corr - tomo, 'corr / tomo image')

## Вот моя попытка отнормировать изображение
Видны следы от прямого пучка (сетка на заднем фоне), но это видимо связано с тем, что прямой пучок зависит от расстояний детектор-источник (сферичность интенсивности), и прямой пучок был померян для другого рассотояния.
К тому-же интенсивнось прямого пучка видимо была меньше (в 16 раз?), чем при проведениии зксперимента. (__это надо проверить__)  

In [ ]:
white_norm = (white - black_1)
white_norm[white_norm<1] = 1
empty_norm = (empty/16 - black_1)
empty_norm[empty_norm<1] =1
my_corr = empty_norm/white_norm
my_corr[my_corr>1.1] = 1.1
show_frame(my_corr, 'my_corr image')

## Скорректированный пучок нами поделеённый на скорреткироваанный скайсканом.
Они вроде совпадают, с точностью до шумов.
Отсюда следует, что нормировка происходит по формуле $$Signal=k\times 2^{16}\frac{I_1-dark}{I_0-dark}, k=0.87$$

In [ ]:
show_frame(my_corr*65535*0.87/corr, 'my_corr/corr image')